# Image Import API - HRFAV Fundus Dataset

This notebook demonstrates how to import images using the `/import/image` api.

For more information: https://eyened.github.io/eyened-platform/getting_started/

The underlying API calls are formulated like this:

1. You need to log in to obtain a session cookie via /api/auth/login-password:

```bash
curl -X POST "http://<host>:<port>/api/auth/login-password" \
  -d '{
    "username": "<admin_username>",
    "password": "<admin_password>",
    "remember_me": false
  }' \
  -c cookies.txt
```  

This stores your session in cookies.txt

2. To add an image, call /api/import/image

```bash
  curl -X POST "http://<host>:<port>/api/import/image" \
  -b cookies.txt \
  -d '{
    "data": <image_payload>,
    "options": {
      "create_project": true,
      "create_patients": true,
      "create_studies": true,
      "create_series": true,
      "include_stack_trace": true
    }
  }'
  ```

  However, this notebooks makes use of the requests library in python and the `ImageImporter` utility class in `eyened_orm.importer.image_importer`

In [1]:
import os
from datetime import date
from pathlib import Path

from dotenv import load_dotenv
from eyened_orm.importer.image_importer import ImageImporter
from tqdm.notebook import tqdm
from utils import download_and_extract_dataset

## Create importer

The importer is a utility class that wraps the API calls using a session for authentication

In [2]:
# for production settings
# env_path = Path.cwd().parent / "orm" / "eyened_orm" / "environments" / "prod_wr.env"
# load_dotenv(env_path)
# importer = ImageImporter(
#     admin_username=os.getenv("ADMIN_USERNAME"),
#     admin_password=os.getenv("ADMIN_PASSWORD"),
#     images_basepath=os.getenv("IMAGES_BASEPATH"),
#     host=os.getenv("HOST"),
#     port=os.getenv("PORT"),
# )

# development settings
env_path = Path.cwd().parent / "dev" / ".env"
load_dotenv(env_path)
importer = ImageImporter(
    admin_username=os.getenv("ADMIN_USERNAME"),
    admin_password=os.getenv("ADMIN_PASSWORD"),
    images_basepath=os.getenv("IMAGES_BASEPATH"),
    host="localhost",
    port=os.getenv("DEV_NGINX_PORT"),
)

## Download and extract HRFAV dataset

In [3]:
extract_dir = Path(importer.images_basepath) / "hrfav"
if not extract_dir.exists():
    os.makedirs(extract_dir, exist_ok=True)
    download_and_extract_dataset(
        "https://www5.cs.fau.de/fileadmin/research/datasets/fundus-images/all.zip",
        extract_dir,
    )

In [4]:
# Set up project information
project_name = "HRFAV"
images_dir = extract_dir / "images"
image_paths = list(images_dir.glob("*.jpg", case_sensitive=False))

print(f"Found {len(image_paths)} images.")

Found 45 images.


## Importing Images 

We'll now import each image individually using the `/import/image` endpoint.

In [5]:
# For tracking results
import_results = []
for idx, img_path in enumerate(tqdm(image_paths, desc="Importing images")):

    # Create the payload

    # important:the path should be relative to the images_basepath
    path = str(img_path.relative_to(importer.images_basepath))
    # just for demo, laterality needs to be set to either "R" or "L" (not the actual laterality for these images)
    laterality = "R" if idx % 2 == 0 else "L"
    # just for demo, we need to indicate a patient_identifier for each image
    patient_identifier = f"Patient_{idx//2}"
    # just for demo, we need to indicate a study_date for each image
    study_date = date.today().isoformat()

    image_payload = {
        "project_name": project_name,
        "patient_identifier": patient_identifier,
        "study_date": study_date,
        "image": path,
        "image_props": {"OldPath": img_path.stem, "Laterality": laterality},
    }

    result = importer.import_image(image_payload)
    import_results.append(result)

    # If an error occurred, print it but continue with the next image
    if not result["success"]:
        print(f"Error importing {img_path.name}: {result['error']}")
        print(result["stack_trace"])
        break

Importing images:   0%|          | 0/45 [00:00<?, ?it/s]

In [6]:
# Summarize the import results
successful_imports = [r for r in import_results if r["success"]]
failed_imports = [r for r in import_results if not r["success"]]

print(f"Total images: {len(image_paths)}")
print(f"Successfully imported: {len(successful_imports)}")
print(f"Failed imports: {len(failed_imports)}")

if failed_imports:
    print("\nFailed imports:")
    for fail in failed_imports:
        print(f"  {fail['image_path']}: {fail['error']}")

Total images: 45
Successfully imported: 45
Failed imports: 0
